In [11]:
#Import required libraries for web scraping
import requests
from bs4 import BeautifulSoup
import pandas as pd

# url of the website
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)

# assign content of the page to BeautifulSoup object
page_content = BeautifulSoup(page.content, 'html.parser')

# extract column information from the 1st table and store it in the list
# print(page_content.find_all('table')[0].find('tr'))
headers = page_content.find_all('table')[0].find('tr')
headers_list = []

for item in headers:
    try:
        headers_list.append(item.get_text().strip('\n'))
    except:
        continue

# print(headers_list)

# extract content of each column and store it in the list
# print(page_content.find_all('table')[0].find_all('tr')[1:])
rows = page_content.find_all('table')[0].find_all('tr')[1:]
rows_list = []


for row in rows:

    row_list = []
    for item in row:
        try:
            row_list.append(item.get_text().strip('\n'))
        except:
            continue

    rows_list.append(row_list)

# print(rows_list)

# dropping all rows with Borough = 'Not assigned'
i = 0
while i < len(rows_list):
    if rows_list[i][1] == 'Not assigned':
        del rows_list[i]
        i -= 1
    i += 1

# assigning Borough to Neighbourhood if Neighbourhood = 'Not assigned'
for i in range(0, len(rows_list)):
    if rows_list[i][2] == 'Not assigned':
        rows_list[i][2] = rows_list[i][1]

    
# creating panda data frame with table information
df_postal_codes = pd.DataFrame(columns=headers_list)
# for i in rows_list:
#     df_postal_codes = df_postal_codes.append(i, ignore_index=True)

for i in range(0, len(rows_list)):
    tmp_row = {'Postal Code':rows_list[i][0], 'Borough':rows_list[i][1], 'Neighbourhood':rows_list[i][2]}
    df_postal_codes = df_postal_codes.append(tmp_row, ignore_index=True)

df_postal_codes.shape

(103, 3)